# RDD Lab

In this lab, we will be working with data from [Libraries.io](http://Libraries.io), a package manager aggregator. Our data consist of two files, one detailing packages and package managers, the other detailing the code repositories the packages are developed in. 

The data is stored as csv files, so to get started import the needed Python packages and read in the data

In [1]:
import csv
from io import StringIO

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17,application_1566055793802_0015,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
text = spark.sparkContext.textFile("hdfs:///data/projects-1.0.0-2017-06-15.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verify the data was read in correctly using take.

In [3]:
text.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID,Platform,Name,Created Timestamp,Updated Timestamp,Description,Keywords,Homepage URL,Licenses,Repository URL,Versions Count,SourceRank,Latest Release Publish Timestamp,Latest Release Number,Package Manager ID,Dependent Projects Count,Language,Status,Last synced Timestamp,Dependent Repositories Count,Repository ID', '1,Alcatraz,21st digital Templates,2015-01-11 23:56:18 UTC,2017-06-14 01:03:14 UTC,"A starting point for stripped down, structured and nib-less iOS applications including support for CocoaPods and Uncrustify.","",https://github.com/21stdigital/Xcode-Templates,"",https://github.com/21stdigital/Xcode-Templates,0,2,2017-06-14 01:03:05 UTC,,,0,Objective-C,,2017-06-14 01:03:10 UTC,0,3945']

You may have noticed that what we read in is a list of strings. Further more, the first string appears to a column headers. 

In the next cell we are going to split each string into a tuple. We have done this step for you, but make sure you can understand what the code below is doing.

In [4]:
data = text.map(lambda x: tuple(next(csv.reader(StringIO(x)))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that we have split each line into tuples, we need to remove the first row, which isn't actually part of the data. 

To do this, use the `filter` function and write a lambda that checks if the first element of the tuple is **not** "ID"

In [5]:
data = data.filter(lambda x: x[0] != "ID")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Once again, use take to look at the format the data is now in

In [6]:
data.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('1', 'Alcatraz', '21st digital Templates', '2015-01-11 23:56:18 UTC', '2017-06-14 01:03:14 UTC', 'A starting point for stripped down, structured and nib-less iOS applications including support for CocoaPods and Uncrustify.', '', 'https://github.com/21stdigital/Xcode-Templates', '', 'https://github.com/21stdigital/Xcode-Templates', '0', '2', '2017-06-14 01:03:05 UTC', '', '', '0', 'Objective-C', '', '2017-06-14 01:03:10 UTC', '0', '3945'), ('2', 'Alcatraz', 'ACCodeSnippetRepository', '2015-01-11 23:56:18 UTC', '2017-06-14 01:03:10 UTC', 'Synchronize code snippets with a git repository.', '', 'https://github.com/acoomans/ACCodeSnippetRepositoryPlugin', 'MIT', 'https://github.com/acoomans/ACCodeSnippetRepositoryPlugin', '0', '11', '2014-03-13 18:40:38 UTC', '0.0.4', '', '0', 'Objective-C', '', '2017-06-14 01:03:07 UTC', '0', '3943')]

The rest of the lab consists of answering questions about the data. 

### How many packages are accounted for in this dataset?

Hint: Use `count`

In [7]:
data.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2215453

### What package managers are included in the data?

The package manager is the second element in each tuple in the RDD, and can be accseed using `tuple[1]`

Hint: Extract the package manager names using `map` and then use `distinct`

In [8]:
managers = data.map(lambda x: x[1]).distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

View the results using `collect`

In [9]:
managers.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Go', 'Sublime', 'Maven', 'Dub', 'NuGet', 'Meteor', 'Hex', 'Pypi', 'Hackage', 'Julia', 'Inqlude', 'Shards', 'Bower', 'Cargo', 'Atom', 'Elm', 'Clojars', 'Rubygems', 'Emacs', 'Nimble', 'Wordpress', 'CPAN', 'Jam', 'Pub', 'PlatformIO', 'Alcatraz', 'NPM', 'Packagist', 'CRAN', 'Homebrew', 'Haxelib', 'CocoaPods', 'Carthage', 'SwiftPM']

### What package has the higest SourceRank?

SourceRank is Libraries.io measure that combines popularity as well as how well maintained the packages is, along with a few other factors

We've done this one for you to show you a unique way to use Python's built in `max` function.

By supplying a key to the `max` function we can compare two tuples using a sepcific element in that tuple. In this case we are comparing source rank, which we can access at the 11th position.

In [10]:
data.reduce(lambda x,y: max(x,y,key=lambda tup: int(tup[11])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('134847', 'NPM', 'chai', '2015-01-27 11:20:15 UTC', '2017-06-14 10:32:21 UTC', 'BDD/TDD assertion library for node.js and the browser. Test framework agnostic.', 'test,assertion,assert,testing,chai', 'http://chaijs.com', 'MIT', 'https://github.com/chaijs/chai', '69', '31', '2017-06-05 19:33:48 UTC', '4.0.2', '', '64779', 'JavaScript', '', '2017-06-14 10:32:21 UTC', '161607', '763')

### What is the most frequent dependency per package manager?

To answer this question, let's break it down into smaller parts. 

First it is a good idea to change the RDD into a Pair RDD, using the package manager as the key. We have done this step for you.

In [11]:
package_manager_keys = data.map(lambda x: (x[1],x))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next, use `reduceByKey` to find the most frequent dependency in each package manager.

Use the built in `max` function from Python, similiarly to as was done above. The element of the tuple you should be comparing on this time is `19`

In [12]:
popular_deps = package_manager_keys.reduceByKey(lambda x,y : max(x , y, key=lambda q: int(q[19])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally, use `map` again to select only the package manager, the package name, and the number of times it is listed as a dependency.

Remember, the RDD elements now have a form of

`(packageMananger, (packageMananger, packageName, .... )`

The packageName can be accessed using `tupleVar[1][2]` and the number of times it is a dependency can be acessed using `tupleVar[1][19]`.


In [13]:
reformatted = popular_deps.map(lambda x: (x[0],x[1][2],x[1][19]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now print the most popular dependencies using `collect`

In [14]:
reformatted.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Go', 'golang.org/x/net/context', '5379'), ('Sublime', 'CMSSWLookup', '0'), ('Maven', 'javax.servlet:servlet-api', '43397'), ('Dub', 'vibe-d', '203'), ('NuGet', 'Newtonsoft.Json', '63559'), ('Meteor', 'meteor', '72'), ('Hex', 'poison', '5950'), ('Pypi', 'requests', '71110'), ('Hackage', '3d-graphics-examples', '0'), ('Julia', 'Compat', '540'), ('Inqlude', 'quickcross', '0'), ('Shards', 'radix', '66'), ('Bower', 'jQuery', '106457'), ('Cargo', 'libc', '6922'), ('Atom', 'laravel-5-snippets', '0'), ('Elm', 'elm-lang/core', '4655'), ('Clojars', 'org.clojure/clojure', '26779'), ('Rubygems', 'rake', '481616'), ('Emacs', 'dtrace-script-mode', '0'), ('Nimble', 'nimrat', '0'), ('Wordpress', 'tradebit-download-shop', '0'), ('CPAN', 'perl', '2723'), ('Jam', 'truncatejs', '0'), ('Pub', 'browser', '2972'), ('PlatformIO', 'OneWire', '0'), ('Alcatraz', '21st digital Templates', '0'), ('NPM', 'express', '380978'), ('Packagist', 'phpunit/phpunit', '122137'), ('CRAN', 'testthat', '6037'), ('Homebrew', 

### Who is the most proflific owner of packages per package manager?

For this next question, we need to consult the second file, which is detailed information about where and who develops each package. Reading in the data will be very similar to as was done above, so we have taken all the steps to split each line and filter the data for you

In [15]:
repos = spark.sparkContext.textFile("hdfs:///data/repositories-1.0.0-2017-06-15.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
repos_data = repos.map(lambda x: tuple(next(csv.reader(StringIO(x),'unix')))).filter(lambda x: x[0] != "ID")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
repos_data.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('1', 'GitHub', 'brianmhunt/knockout-modal', 'Opinionated modals with Knockout.js', 'false', '2014-09-15 01:21:34 UTC', '2016-12-28 16:33:17 UTC', '2016-12-18 18:31:32 UTC', 'http://brianmhunt.github.io/knockout-modal/', '512', '7', 'JavaScript', 'true', 'true', 'true', '0', '', '1', 'master', '2', '24038237', '', 'MIT', '1', 'README.md', '', '', 'LICENSE', '', '', '', '', '2016-05-27 15:42:48 UTC', '5', 'GitHub', '', '', '', '', ''), ('2', 'GitHub', 'SteveSanderson/knockout.mapping', 'Object mapping plugin for KnockoutJS', 'false', '2010-11-01 09:27:43 UTC', '2016-12-28 16:45:05 UTC', '2016-10-11 01:58:14 UTC', '', '924', '535', 'JavaScript', 'true', 'true', 'false', '708', '', '85', 'master', '61', '1041356', '', 'MIT', '21', 'README.md', '', '', 'LICENSE', '', '', '', '', '2016-05-27 15:21:05 UTC', '9', 'GitHub', '', '', '', '', '')]

To answer this question we are going to join our two RDDs together. In order to do that, first we need to once again convert them into a Pair RDD, this time with the primary key from the repository dataset

Use `map` to make a (key,value) pair for each element in the `repos_data` RDD. The tuple index for the key is `0`

In [18]:
repos_to_join = repos_data.map(lambda x: (x[0],x))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to make a (key, value) pair for each element in the `data` RDD. The tuple index for the key is `-1`

In [19]:
data_to_join = data.map(lambda x: (x[-1],x))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `join` to join the two RDDs together into a single RDD

In [20]:
joined = data_to_join.join(repos_to_join)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that we have a single RDD, we can prepare to count the number of packages each owner has in each package manager. We have done this step for you. 

What the code below is doing is creating a tuple of the form

`((packageManager, repositoryOwner), 1)`

In [21]:
package_owners = joined.map(lambda x: ((x[1][0][1],x[1][1][2].split('/')[0]),1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `reduceByKey` to add up the total number for reach `packageManager,reposititoryOwner` pair

In [22]:
counts = package_owners.reduceByKey(lambda x,y: x + y)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to reformat the data so just the package manager is the key now.

The data looked like 

`((packageManager, repositoryOwner), count)` before,

but now it should looke like

`(packageManager, (repositoryOwner, count))`

In [23]:
counts_reformated = counts.map(lambda x: (x[0][0],(x[0][1],x[1])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that the data is in the right format, use `reduceByKey` and `max` to find the most prominent owner per package manager.

Hint: remember that the `max` function in python can take a `key` value

In [24]:
maxes = counts_reformated.reduceByKey(lambda x,y: max(x , y, key=lambda val: int(val[1])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to reformat the data one more time, into a 3-tuple per element.

When you are done, your data should look like

`(packageMananger, repositoryOwner, count)`

In [25]:
maxes_reformatted = maxes.map(lambda x: (x[0],x[1][0],x[1][1]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally, use `collect` to display the results

In [26]:
maxes_reformatted.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('CocoaPods', 'hyperoslo', 50), ('CPAN', 'perlancar', 1204), ('Hackage', 'brendanhay', 208), ('Sublime', 'SublimeLinter', 36), ('SwiftPM', 'ZewoGraveyard', 85), ('Haxelib', 'haxe-react', 35), ('Emacs', 'syohex', 45), ('Inqlude', 'cneben', 3), ('Alcatraz', 'johnno1962', 7), ('Cargo', 'retep998', 428), ('Carthage', 'wireapp', 49), ('Pub', 'dart-lang', 246), ('Julia', 'JuliaStats', 28), ('Homebrew', 'google', 19), ('Meteor', 'rzymek', 110), ('Jam', 'aureooms', 56), ('Bower', 'google-fonts-bower', 997), ('Atom', 'phthhieu', 133), ('NPM', 'npmdoc', 5207), ('Wordpress', 'claudiosanches', 18), ('Rubygems', 'jrobertson', 303), ('PlatformIO', 'adafruit', 173), ('Packagist', 'thecodingmachine', 212), ('Go', 'cloudfoundry', 2007), ('NuGet', 'aws', 360), ('Nimble', 'achesak', 39), ('Dub', 'DerelictOrg', 25), ('Maven', 'kiegroup', 980), ('Clojars', 'cljsjs', 238), ('Elm', 'elm-community', 51), ('Shards', 'ysbaddaden', 7), ('Hex', 'nerves-project', 35), ('CRAN', 'ropensci', 117), ('Pypi', 'collecti

### What is the correlation between number of github stars and number of times a package is listed as a dependency?

Once again, we will be working with the joined RDD.

First we need to retrieve the two pieces of information we need from each element in the RDD.

Because some of the elements of the RDD don't have this information, we have written the function below to assign 0 to missing values

In [27]:
def turn_to_int(string):
    try:
        return int(string)
    except:
        return 0

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run the code below to extract the two numbers we need

In [28]:
stars_and_deps = joined.map(lambda x: (turn_to_int(x[1][0][19]),turn_to_int(x[1][1][10])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now we will use reduce to add up the number of stars and dependencies. We have done this step for you because calling reduce on tuples can be tricky.

In [29]:
mean_stars = stars_and_deps.reduce(lambda x,y: (x[0] + y[0],))[0]/stars_and_deps.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
mean_deps = stars_and_deps.reduce(lambda x,y: (1,x[1] + y[1]))[1]/stars_and_deps.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to calculate the errors for each element in the RDD, by subtracting either `mean_stars`, or `mean_deps` from the appropriate value in the tuple

In [31]:
errors = stars_and_deps.map(lambda x: (x[0] - mean_stars, x[1] - mean_deps))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to square each value of the tuple

In [32]:
sq_errors = errors.map(lambda x: (x[0]**2, x[1]**2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `reduce` to calculate the sums of the squared errors

In [33]:
sums_of_squares = sq_errors.reduce(lambda x,y: (x[0] + y[0], x[1] + y[1]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Get the denominator by taking the square root of each sum and multiplying them together

In [34]:
import math

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
denominator = math.sqrt(sums_of_squares[0]) *  math.sqrt(sums_of_squares[1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next use `map` to multiply together the error for the stars and the error for the dependencies for each element in the RDD

In [36]:
products = errors.map(lambda x: x[0] * x[1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Get the numerator of the equation by using `reduce` to sum all the products from the previous cell

In [37]:
numerator = products.reduce(lambda x,y:x + y)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally, get the correlation by dividing the numerator by the denominator

In [38]:
numerator /  denominator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.06705309395947452

### Which package names are found in both npm and pypi
For the final question, we are going to use set operations.

First we need to find the packages in pypi and the packages in NPM

Use `filter` to find all the elements of the RDD whose first value is equal to "Pypi"

In [39]:
pypi = data.filter(lambda x: x[1] == "Pypi")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `filter` to find all the elements of the RDD whose first value is equal to "NPM"

In [40]:
npm = data.filter(lambda x: x[1] == "NPM")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to get an RDD of only the names of the pypi packages. The names are the 2nd value of the tuple

In [41]:
pypi_names = pypi.map(lambda x: x[2])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `map` to get an RDD of only the names of the npm packages. The names are the 2nd value of the tuple

In [42]:
npm_names = npm.map(lambda x: x[2])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `intersection` to get the names that appear in both

In [43]:
intersection = pypi_names.intersection(npm_names)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

View the names that appear in both by calling `collect`

In [44]:
intersection.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['aaa', 'accessibility', 'aerospike', 'aes', 'affinity', 'afn', 'agentx', 'airbnb', 'algae', 'algorithms', 'aloha', 'antigravity', 'ape', 'apiclient', 'appy', 'apricot', 'argus', 'ari', 'asciiart', 'aspect', 'assemblage', 'asset', 'atl', 'aubio', 'autogit', 'axon', 'b3', 'baf', 'bam', 'basicauth', 'basin', 'batchy', 'beanbag', 'beatport', 'bebop', 'beep', 'behave', 'bento', 'bird', 'bisheng', 'bitarray', 'bladerunner', 'blast', 'blazar', 'blocks', 'boa', 'book', 'bookends', 'boomslang', 'boss', 'boundary', 'brace', 'browserstack', 'bruce', 'bureaucrat', 'butterfly', 'cash', 'cassowary', 'cat', 'ccp', 'celery', 'census', 'chardet', 'checker', 'checkers', 'ckan', 'clearance', 'clog', 'closure', 'clusto', 'cocaine', 'cocoon', 'colorlab', 'common', 'community', 'concepts', 'conduit', 'cormoran', 'corrector', 'cottontail', 'couchexport', 'crab', 'crone', 'cs', 'ctable', 'ctp', 'cute', 'dae', 'dalmatian', 'datab', 'datasift', 'datomic', 'datrie', 'debugger', 'dec', 'declare', 'defensio', 'de